In [247]:
from numba import cuda
from numba import float32,int32,int64
import numpy as np
import pdb
from math import floor
from decimal import *

# Sympy stuff for checking
from sympy import Poly, expand 
from sympy.abc import x, y, u, v


def parsePolynomialString(polyString):
    
    
    try:
        polyString = unicode(polyString,"utf-8")
    except TypeError: #check if already unicode
        pass
    
    repls = ('+', '!'), ('-', '!')
    temp = [l.strip() for l in reduce(lambda a, kv: a.replace(*kv), repls, polyString ).split('!')]
    temp2 = np.asarray([split_coefficient_variable(t) for t in temp])
    temp2[:,0] = map(lambda x: u'1' if x==u'' else x,temp2[:,0])
    
    signs = ''.join(ch for ch in polyString if ch == '+' or ch == '-')
    if len(signs)==len(temp2)-1:
        signs = '+'+signs
    signs = np.asarray([c for c in signs])
    
    
    coeffs = [int(s+val) for (s,val) in zip(signs,temp2[:,0])]
    variables = temp2[:,1]
    
    return coeffs,variables

def split_coefficient_variable(string):
    n = len(string)
    for ind,c in enumerate(string):
        if c.isnumeric():
            if ind == n-1:
                return (string,u'0')
            else:
                continue
        else:
            return (string[0:ind],string[ind+1::])
        
def findDegreeUnivariate(v):
    end_ind = -1 # a hack way for a flag
    start_ind = -1
    for i,c in enumerate(v):
        if c.isnumeric() and start_ind == -1:
            start_ind = i
            if start_ind == len(v) -1:
                end_ind = start_ind
                break
        elif (not c.isnumeric() and end_ind == -1 and not start_ind  == -1) or i == len(v)-1:
            end_ind = i
            break

        else:
            continue
    #pdb.set_trace()
    if not start_ind == end_ind:
         if end_ind == 0 and start_ind == -1:
            return 1
         else:
            t = v[start_ind:end_ind+1]
       
    else:
        t = v[start_ind]
    
    return int(t)

def buildDegrees(varsfull):
    ''' Simple wrapper function that builds a list of list for all the degrees of each univariate variable '''
    degreeLists = [[findDegreeUnivariate(var) for var in vars.split()] for vars in varsfull]
    return degreeLists


def primesfrom3to(n):
    """ Returns a array of primes, 3 <= p < n """
    sieve = np.ones(n/2, dtype=np.bool)
    for i in xrange(3,int(n**0.5)+1,2):
        if sieve[i/2]:
            sieve[i*i/2::i] = False
    return 2*np.nonzero(sieve)[0][1::]+1
    
def get_mod_primes(N,M):
    primes = primesfrom3to(N)
    primes = primes.tolist()
    total = 1L
    mvect = np.array([],dtype=np.int32)
    for p in primes:
        total *= p
        mvect = np.append(mvect,p)
        #print total
        if total > M:
            return mvect
    return mvect

def generate_evaluationpoints(degreeResult):
    return np.arange(1,degreeResult)


def newton_interp_univariate(x,b,d):
    # x: set of points
    # b: corresponding value evaluated at the point
    # d: degree of polynomial (should be equal to len(x)-1
    n = len(x)
    V = np.array([x**i for i in range(d+1)][::-1]).T
    A = np.dot(np.linalg.inv(V),b)
    return A


def EGCD(a,b):
    cc = a
    dd = b
    c1 = 1
    c2 =0
    d1 =0
    d2 = 1
    
    while not dd==0:
        try:
            q = cc/dd
        except TypeError:
            pdb.set_trace()
            
        r = cc - int(q*dd)
        r = int(r)
        cc = dd
        dd = r

        r1 = c1 - int(q*d1)
        r1 = int(r1)
        c1 = d1
        d1 = r1

        r2 = c2 - int(q*d2)
        r2 = int(r2)
        c2 = d2
        d2 = r2
    
    return (cc,c1,c2)

def cra_incremental(rvect,mvect):
    k = len(rvect)
    assert(len(rvect) == len(mvect))
    
    M = mvect[0]
    res = rvect[0]
    M_invs = []
    for i in range(1,k):
        M_inv = EGCD(M,mvect[i])[1] 
        if M_inv == 0: pdb.set_trace()
        M_invs.append(M_inv)
        c = int(M_inv%mvect[i])
        rprime = int(res%mvect[i])
        s = c*(rvect[i] - rprime)#%mvect[i]
        s = int(s)
        res = int(res+s*M)
        #import pdb; pdb.set_trace()
        M = int(M*mvect[i])
    return res

def MRC_alg(rvect,mvect,c):
    k = len(rvect)
    gamma = np.empty(k,dtype = np.int32)
    gamma[0] = rvect[0]
    #gamma[0] = rvect[0]
    M = np.ones(k,dtype=np.int32)

    for i in range(1,k):
        gamma[i] = ((rvect[i] - gamma[0])*c[i]) % mvect[i]
        M[i] = (mvect[0]*c[i]) % mvect[i]

    for i in range(1,k-1):
        for j in range(i+1,k):
            gamma[j] = (gamma[j] - gamma[i]*M[j]) % mvect[j]
            M[j] = (M[j]*mvect[i]) % mvect[j]
    return gamma,M

def homer_scheme(mvect,gammas,i):
    if i==len(mvect):
        return 1
    return gammas[i]+mvect[i]*(homer_scheme(mvect,gammas,i+1))

def symmetric_residue(r,m):
    if r <= m // 2: # divide by 2
        return r
    return r - m


@cuda.jit(argtypes=[int32[:],int32[:],int32,int32,int32,int32,int32], target='gpu')
def evaluate_polynomial(a,c,n,d,k,l,alpha_os):
    i = cuda.grid(1)
    N = n*d*k*l
    #we need a representation for the polynomial (x^(p-1),x^p,...x^0)
    
    if i<N:

        alpha = (i%(n*d))/d
        #alpha = (i%6)/d
        #alpha = (i%(12))/d
        degree = i%d
        ind = degree #it just so happens that the degree is equal to the index in this case
        offset = (i/(n*d)*d)
        #cuda.syncthreads() # remind myself for shared memory implemtation
        c[i] = a[ind+offset]*(alpha+alpha_os)**degree
        
    

In [288]:
print n
d = d_test
print d

5
3


In [294]:
45%15

0

In [350]:
c[50]

1

In [295]:
14+15+15

44

In [293]:
(n*d)

15

In [375]:
c[89]

16

In [377]:
i=89
print (i/(n*d)*d)
print (i%(n*d))/d
print i%d + (i/(n*d)*d) 

16
4
17


In [383]:
ab

array([2, 1, 1, 0, 1, 1, 5, 1, 1, 0, 2, 1, 2, 0, 1, 4, 5, 1])

In [380]:
ab

array([2, 1, 1, 0, 1, 1, 5, 1, 1, 0, 2, 1, 2, 0, 1, 4, 5, 1])

In [379]:
c[89]

16

In [ ]:
ab[4]

In [343]:
ab[9:]

array([0, 2, 1, 2, 0, 1, 4, 5, 1])

In [336]:
i/(n*d)

2

In [338]:
ab

array([2, 1, 1, 0, 1, 1, 5, 1, 1, 0, 2, 1, 2, 0, 1, 4, 5, 1])

In [334]:
ab[6]

5

In [323]:
c[49]

2

In [279]:
c[44]

16

In [278]:
90/15

6

In [276]:
N

90

In [277]:
ab

array([2, 1, 1, 0, 1, 1, 5, 1, 1, 0, 2, 1, 2, 0, 1, 4, 5, 1])

In [274]:
c[44:47]

array([16,  0,  0])

In [270]:
14+15*2

44

In [113]:
3%24/2

1

In [114]:
3%24/2

1

In [69]:
a = 'x + 5'
b = '5 x - 3'
aCoef,aVars = parsePolynomialString(a)
bCoef,bVars = parsePolynomialString(b)
ab = np.hstack([np.hstack([np.mod(_Coef[::-1],mi) for mi in m]) for _Coef in [aCoef,bCoef]])
print ab
print aCoef[::-1],

[2 1 0 1 5 1 0 2 2 0 4 5]


[5, 1]

In [70]:
a = 'x^2 + x + 5'
b = 'x^2 + 5 x - 3'
aCoef,aVars = parsePolynomialString(a)
bCoef,bVars = parsePolynomialString(b)
ab = np.hstack([np.hstack([np.mod(_Coef[::-1],mi) for mi in m]) for _Coef in [aCoef,bCoef]])
print ab
aCoef[::-1]


[2 1 1 0 1 1 5 1 1 0 2 1 2 0 1 4 5 1]


[5, 1, 1]

In [141]:
2**81

2417851639229258349412352L

In [72]:
''' Set up CUDA parameters'''
#a = np.hstack([a_5[::-1],a_7[::-1]])
bpg = 50
tpb = 16
deg = cDegree
k=len(m)
n=3 # the number of evaluation points
l=2  # the number of polies
alphas = np.arange(n)

N = deg*k*n*l

c = np.empty(N,dtype=np.int32)
evaluate_polynomial[bpg,tpb](ab,c,n,deg,k,l)


## Infinite Precision CSA

In [2]:
    '''Lets take a look at some modular arthimitic'''
num = 515033918625080621034667968750231312512515215125215125125
print "num has {} digits ".format(len(str(num)))

mvect = get_mod_primes(10000,num).tolist()
print "{} primes were required to reconstruct num\n".format(len(mvect))
residues =[int(num%m) for m in mvect] 
c = [0] + [EGCD(reduce(lambda x,y:x*y,mvect[0:k+1]),m)[1] for (k,m) in enumerate(mvect[1:])]
vv,M = MRC_alg(residues,mvect,c)
vv = vv.tolist()
M = M.tolist()

print "v: {} \n".format(vv)

u = homer_scheme(mvect,vv,0)%reduce(lambda x,y: x*y,mvect)

mvect2 = get_mod_primes(10000,u).tolist()
print "Solution Residues: {} \n".format(residues)
residues2 =[int(u%m) for m in mvect2]
print "Check Residues: {}\n".format(residues)


num has 57 digits 
34 primes were required to reconstruct num

v: [2, 1, 0, 4, 4, 11, 3, 0, 11, 11, 3, 18, 42, 36, 50, 48, 12, 27, 4, 27, 6, 4, 59, 49, 50, 4, 68, 57, 19, 124, 46, 0, 119, 102] 

Solution Residues: [2, 0, 5, 7, 1, 6, 16, 13, 21, 21, 1, 28, 9, 9, 49, 37, 49, 33, 33, 60, 16, 40, 5, 59, 22, 22, 10, 33, 52, 79, 114, 50, 119, 112] 

Check Residues: [2, 0, 5, 7, 1, 6, 16, 13, 21, 21, 1, 28, 9, 9, 49, 37, 49, 33, 33, 60, 16, 40, 5, 59, 22, 22, 10, 33, 52, 79, 114, 50, 119, 112]



16

In [50]:
N/12

6

In [77]:
'''Reconstruct and interpolate'''
X = np.reshape(c,((N/deg),deg))
X = np.sum(X,axis=1)
X = np.reshape(X,(k*n,l),order='F')
X  = np.product(X,axis=1)
X = np.reshape(X,(k,n))
X

array([[  4,  28, 304],
       [  5,  48, 217],
       [  0,   0,   0]])

In [355]:
c[30]

5

In [37]:
72/4

18

In [352]:
M = 2*int(max(np.abs(aCoef)))*int(max(np.abs(bCoef)) )
M = 2*M
print M

100


## Polynomial Multplication

In [416]:
'Univariate Case'
from sympy.abc import x, y, u, v


#a = 'x^2 + x + 5'
#b = 'x^2 + 5 x - 3'
a_sym = Poly('x**2 + x + 5',x)
b_sym = Poly('x**2 + 5*x - 3',x)

a = '7 x + 5'
b = '2 x - 3'
aCoef,aVars = parsePolynomialString(a)
bCoef,bVars = parsePolynomialString(b)

aDegree = max([np.sum(_) for _ in buildDegrees(aVars)])
bDegree = max([np.sum(_) for _ in buildDegrees(bVars)])
cDegree = aDegree+bDegree

''' setup '''
M = 2*int(max(np.abs(aCoef)))*int(max(np.abs(bCoef)) )
M = 2*M
print M
#Calculate homomorphisms
# syntax: x_'i' where i corresponds to mod reduction and x is the polynomial in question i.e (a,b)
# for this example we use m = 5,7
m = get_mod_primes(100,M)
#m = [5,7]
is_symmetric = True
''' reduce polynomials'''
ab = np.hstack([np.hstack([np.mod(_Coef[::-1],mi) for mi in m]) for _Coef in [aCoef,bCoef]])

''' Set up CUDA parameters'''
#a = np.hstack([a_5[::-1],a_7[::-1]])
bpg = 50
tpb = 16
deg = cDegree
k=len(m)
n=deg+1 # the number of evaluation points
l=2  # the number of polies
alpha_os = 0
alphas = np.arange(n)+alpha_os
d_test = aDegree+1
N = d_test*k*n*l

c = np.empty(N,dtype=np.int32)

evaluate_polynomial[bpg,tpb](ab,c,n,d_test,k,l,alpha_os)

'''Reconstruct and interpolate'''
X = np.reshape(c,((N/d_test),d_test))
X = np.sum(X,axis=1)
X = np.reshape(X,(k*n,l),order='F')
X  = np.product(X,axis=1)
X = np.reshape(X,(k,n)) # final 

from sympy.polys.polyfuncs import interpolate
from sympy.abc import x

Y = np.empty_like(X)
for i,row in enumerate(X):
    Y[i,:] = newton_interp_univariate(x=alphas,b=row,d=deg)%m[i]
    #Y[i,:] = Poly(interpolate(row,x)).all_coeffs()
   
gamma = [0] + [EGCD(reduce(lambda x,y:x*y,m[0:k_+1]),mi)[1] for (k_,mi) in enumerate(m[1:])]
d=2
u = np.zeros(deg+1)
for i,row in enumerate(Y.T):
    vv,M = MRC_alg(row,m,gamma)
    G = reduce(lambda x,y: x*y,m)
    temp = homer_scheme(m,vv,0)%G
    if is_symmetric:
        u[i] = symmetric_residue(temp,G)
    else:
        u[i] = temp
    
print "The coefficients are {}".format(u)

var_list = ['{}x^{}'.format(int(cf),p) for cf,p in zip(u,range(len(u))[::-1])]
print "The full poly is: {}".format(reduce(lambda x,y:x+' '+y,var_list))
print "Sympy's Solution is: {}".format(a_sym.mul(b_sym))

84
The coefficients are [ 14. -11. -15.]
The full poly is: 14x^2 -11x^1 -15x^0
Sympy's Solution is: Poly(x**4 + 6*x**3 + 7*x**2 + 22*x - 15, x, domain='ZZ')


In [418]:
Poly(interpolate(row,x)).coeffs()

[3, -9, 6]

In [420]:
Poly(interpolate(row,x))



Poly(3*x**2 - 9*x + 6, x, domain='ZZ')

[1, -1, 1, 1, -2]

In [391]:
from sympy.polys.polyfuncs import interpolate
from sympy.abc import x


x**4 - x**3 + x**2 + x - 2

In [262]:
print a
print b

x^2 + x + 5
x^2 + 5 x - 3


In [265]:
d_test

3

In [260]:
c

array([ 2,  0,  0,  2,  1,  1,  2,  2,  4,  2,  3,  9,  2,  4, 16,  0,  0,
        0,  0,  1,  1,  0,  2,  4,  0,  3,  9,  0,  4, 16,  5,  0,  0,  5,
        1,  1,  5,  2,  4,  5,  3,  9,  5,  4, 16,  0,  0,  0,  0,  2,  1,
        0,  4,  4,  0,  6,  9,  0,  8, 16,  2,  0,  0,  2,  0,  1,  2,  0,
        4,  2,  0,  9,  2,  0, 16,  4,  0,  0,  4,  5,  1,  4, 10,  4,  4,
       15,  9,  4, 20, 16])

In [230]:
'Univariate Case'
from sympy.abc import x, y, u, v


a = 'x^2 + x + 5'
b = 'x^2 + 5 x - 3'
a_sym = Poly('x**2 + x + 5',x)
b_sym = Poly('x**2 + 5*x - 3',x)

a = '7 x + 5'
b = '2 x - 3'
aCoef,aVars = parsePolynomialString(a)
bCoef,bVars = parsePolynomialString(b)

aDegree = max([np.sum(_) for _ in buildDegrees(aVars)])
bDegree = max([np.sum(_) for _ in buildDegrees(bVars)])
cDegree = aDegree+bDegree

''' setup '''
M = 2*int(max(np.abs(aCoef)))*int(max(np.abs(bCoef)) )
M = 2*M
print M
#Calculate homomorphisms
# syntax: x_'i' where i corresponds to mod reduction and x is the polynomial in question i.e (a,b)
# for this example we use m = 5,7
m = get_mod_primes(100,M)
m = [5,7]
is_symmetric = True
''' reduce polynomials'''
ab = np.hstack([np.hstack([np.mod(_Coef[::-1],mi) for mi in m]) for _Coef in [aCoef,bCoef]])

''' Set up CUDA parameters'''
#a = np.hstack([a_5[::-1],a_7[::-1]])
bpg = 50
tpb = 16
deg = cDegree
k=len(m)
n=deg+1 # the number of evaluation points
l=2  # the number of polies
alphas = np.arange(n)
d_test = aDegree+1
N = d_test*k*n*l

c = np.empty(N,dtype=np.int32)

evaluate_polynomial[bpg,tpb](ab,c,n,d_test,k,l)

'''Reconstruct and interpolate'''
X = np.reshape(c,((N/d_test),d_test))
X = np.sum(X,axis=1)
X = np.reshape(X,(k*n,l),order='F')
X  = np.product(X,axis=1)
X = np.reshape(X,(k,n)) # final 


Y = np.empty_like(X)
for i,row in enumerate(X):
    Y[i,:] = newton_interp_univariate(x=alphas,b=row,d=deg)%m[i]
   
gamma = [0] + [EGCD(reduce(lambda x,y:x*y,m[0:k_+1]),mi)[1] for (k_,mi) in enumerate(m[1:])]
d=2
u = np.zeros(deg+1)
for i,row in enumerate(Y.T):
    vv,M = MRC_alg(row,m,gamma)
    G = reduce(lambda x,y: x*y,m)
    temp = homer_scheme(m,vv,0)%G
    if is_symmetric:
        u[i] = symmetric_residue(temp,G)
    else:
        u[i] = temp
    
print "The coefficients are {}".format(u)

var_list = ['{}x^{}'.format(int(cf),p) for cf,p in zip(u,range(len(u))[::-1])]
print "The full poly is: {}".format(reduce(lambda x,y:x+' '+y,var_list))
print "Sympy's Solution is: {}".format(a_sym.mul(b_sym))

3

In [199]:
d_test

2

In [167]:
ab

array([0, 2, 5, 0, 2, 2, 4, 2])

In [163]:
c[23]

4

In [134]:
print alphas
row

[0 1 2 3 4]


array([   4,   28,  304, 1666, 6028])

In [133]:
2**81

2417851639229258349412352L

In [ ]:
print a_sym.mul(b_sym)

In [ ]:

a_t = Poly('5*x + 3',x,y)


In [ ]:
a_t.mul(Poly('5*x + 3',x,y))

In [ ]:



X = np.reshape(c,((N/deg),deg))
X = np.sum(X,axis=1)
X = np.reshape(X,(k*n,l),order='F')
X  = np.product(X,axis=1)
X = np.reshape(X,(k,n))